In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image_dataset_from_directory


In [2]:
from google.colab import files
files.upload()

Saving archive.zip to archive.zip


In [5]:
!unzip archive.zip

Archive:  archive.zip
replace Cityscape Dataset/Bridge/001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [33]:

data_path = "/content/Cityscape Dataset/"

dataset = image_dataset_from_directory(data_path,
                                       image_size=(128, 128),
                                       batch_size=125,
                                       label_mode='int')

class_names = dataset.class_names

Found 8000 files belonging to 10 classes.


In [34]:


# for images, labels in dataset.take(1):
#     plt.figure(figsize=(10, 10))
#     for i in range(9):
#         ax = plt.subplot(3, 3, i + 1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_names[labels[i]])
#         plt.axis("off")


In [35]:
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.cast(label, tf.int64)
    return image, label

dataset = dataset.map(normalize)


In [36]:
num_batches = 8
train_size = int(0.8 * num_batches)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)


In [37]:
for images, labels in train_dataset.take(1):
  print(images.shape)
  print(labels.shape)

(125, 128, 128, 3)
(125,)


In [38]:
for images, labels in test_dataset.take(1):
  print(images.shape)
  print(labels.shape)

(125, 128, 128, 3)
(125,)


In [47]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), input_shape=(128,128, 3), activation='relu', padding='same'),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPool2D(pool_size=2, strides=2),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPool2D(pool_size=2, strides=2),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPool2D(pool_size=2, strides=2),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation='tanh'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='tanh'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])

model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_30 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 conv2d_31 (Conv2D)          (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 64, 64, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_10 (Dropout)        (None, 64, 64, 32)        0         
                                                                 
 conv2d_32 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 conv2d_33 (Conv2D)          (None, 64, 64, 64)        36928     
                                                      

In [48]:
my_optimizer = optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=my_optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy', 'mse']
)


In [49]:
epochs_count = 50
model.fit(train_dataset, epochs=epochs_count)


Epoch 1/50
6/6 [==============================] - 4s 205ms/step - loss: 2.4198 - accuracy: 0.0960 - mse: 28.0260
Epoch 2/50
6/6 [==============================] - 2s 191ms/step - loss: 2.3023 - accuracy: 0.1280 - mse: 28.9465
Epoch 3/50
6/6 [==============================] - 2s 189ms/step - loss: 2.2955 - accuracy: 0.1147 - mse: 27.5050
Epoch 4/50
6/6 [==============================] - 2s 196ms/step - loss: 2.2616 - accuracy: 0.1387 - mse: 28.4129
Epoch 5/50
6/6 [==============================] - 2s 188ms/step - loss: 2.2396 - accuracy: 0.1600 - mse: 29.1181
Epoch 6/50
6/6 [==============================] - 2s 191ms/step - loss: 2.2250 - accuracy: 0.1627 - mse: 27.9645
Epoch 7/50
6/6 [==============================] - 2s 192ms/step - loss: 2.1692 - accuracy: 0.1947 - mse: 28.7768
Epoch 8/50
6/6 [==============================] - 2s 189ms/step - loss: 2.1644 - accuracy: 0.2040 - mse: 27.8817
Epoch 9/50
6/6 [==============================] - 2s 193ms/step - loss: 2.0939 - accuracy: 0.236

In [50]:
model.evaluate(test_dataset)

58/58 [==============================] - 6s 82ms/step - loss: 1.4488 - accuracy: 0.5710 - mse: 27.5520


[1.4487961530685425, 0.5710344910621643, 27.552040100097656]

In [1]:
for images, labels in test_dataset.take(1):
  predictions = model.predict(images)
  predicted_labels = tf.argmax(predictions, axis=1)

  for i in range(len(images)):
    print(f"Predicted: {class_names[predicted_labels[i]]}, Actual: {class_names[labels[i]]}")

NameError: name 'test_dataset' is not defined